In [1]:
""" External Imports """
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import sys
import yaml
import numpy as np
sys.path.append("/Users/emilieyu/endotehelial-masboss")

ipylab module is not installed, menus and toolbar are disabled.


In [2]:
""" Packge Imports """
from boolean_models.analysis import (
    compute_delta,
    classify_phenotype,
    save_df_to_csv
)
from boolean_models.scripts.run_perturbations import run_perturbations
from boolean_models.scripts.run_param_sweep import run_param_sweep_single


In [3]:

""" Global Setting Configuration """

PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
CONFIG_PATH = PROJECT_ROOT / "config" / "rho_sim_config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# MaBoSS Model files
MODELS_BND = PROJECT_ROOT / config['paths']['model_bnd']
MODELS_CFG = PROJECT_ROOT / config['paths']['model_cfg']

# Result directory
RESULTS_DIR = PROJECT_ROOT / config['paths']['results_base']

# Subdirectories to store parameter sweep / perturbation result
PARAM_DIR = RESULTS_DIR / config['paths']['subdirs']['param_sweep']
PERBS_DIR = RESULTS_DIR / config['paths']['subdirs']['perturbation_sim']

# Perturbations
PERBS_DICT = config['perturbations'] # get mutations directly from config
PARAM_LIST = config['sensitivity_analysis'] # get list of parameters

In [4]:
base_model = maboss.load(str(MODELS_BND), str(MODELS_CFG))

In [5]:
perb_df = run_perturbations(base_model, PERBS_DIR, config)
perb_df

DEUBG: Loaded base MaBoSS model
DEBUG: Running scenario: WT
DEBUG: Running scenario: DSP
DEBUG: Running scenario: TJP1
DEBUG: Running scenario: JCAD
DEBUG: Running scenario: DSP_JCAD
DEBUG: Running scenario: TJP1_JCAD
DEBUG: All simulations completed successfully
DEBUG: File perturbation_timeseries_02.csv successfully written to directory: /Users/emilieyu/endotehelial-masboss/results/boolean_models/perturbation_sim


,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
0,0.0,0.502600,0.501000,0.507700,0.530666,0.537277,WT,0.006611,Normal
1,0.1,0.502599,0.500999,0.507698,0.569859,0.576336,WT,0.006477,Normal
2,0.2,0.502600,0.501000,0.507699,0.583240,0.589919,WT,0.006679,Normal
3,0.3,0.502600,0.500998,0.507699,0.589692,0.592001,WT,0.002309,Normal
4,0.4,0.502601,0.500999,0.507700,0.592736,0.595340,WT,0.002604,Normal
...,...,...,...,...,...,...,...,...,...
235,3.5,0.505000,0.000000,0.000000,0.651804,0.420171,TJP1_JCAD,-0.231633,Normal
236,3.6,0.505000,0.000000,0.000000,0.654923,0.418987,TJP1_JCAD,-0.235936,Normal
237,3.7,0.505000,0.000000,0.000000,0.657327,0.420278,TJP1_JCAD,-0.237049,Normal
238,3.8,0.505001,0.000000,0.000000,0.657322,0.417166,TJP1_JCAD,-0.240156,Normal


In [6]:
dsp_df = perb_df[perb_df['perturbation'] == 'DSP']
dsp_df

,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
40,0.0,0.0,0.495699,0.499700,0.488741,0.537587,DSP,0.048846,Normal
41,0.1,0.0,0.495700,0.499700,0.460624,0.580811,DSP,0.120187,Normal
42,0.2,0.0,0.495700,0.499700,0.444585,0.608223,DSP,0.163638,Normal
43,0.3,0.0,0.495700,0.499700,0.432502,0.625374,DSP,0.192872,Normal
44,0.4,0.0,0.495700,0.499698,0.421166,0.635658,DSP,0.214492,Normal
45,0.5,0.0,0.495699,0.499699,0.417114,0.643842,DSP,0.226728,Normal
46,0.6,0.0,0.495699,0.499700,0.414028,0.648627,DSP,0.234599,Normal
47,0.7,0.0,0.495699,0.499700,0.410828,0.650045,DSP,0.239217,Normal
48,0.8,0.0,0.495700,0.499700,0.409578,0.654308,DSP,0.244730,Normal
49,0.9,0.0,0.495699,0.499700,0.407493,0.659260,DSP,0.251767,Hyper
